# Example of training an INR locally
This notebook provides an example of how to create an INR and train it locally using the tools in this repo.

In [1]:
import pdb
import traceback

import jax
import optax
import wandb

from common_dl_utils.config_creation import Config
import common_jax_utils as cju

wandb.login()

key = jax.random.PRNGKey(12398)
key_gen = cju.key_generator(key)

wandb: Currently logged in as: maxwell_litsios (INR_NTK). Use `wandb login --relogin` to force relogin


# Note
It seems to work, but we might want to implement gradient accumulation into the training loop so we can use larger batch sizes.

Also, it would be good to implement metrics.

We want to train a single INR on `example_data/parrot.png`. We'll use the `CombinedINR` clas from `model_components.inr_modules` together with the `SirenLayer` and `GaussianINRLayer` from `model_components.inr_layers` for the model, and we'll train it using the tools from `inr_utils`.

To do all of this, basically we only need to create a config. We'll use the `common_dl_utils.config_creation.Config` class for this, but this is basically just a dictionary that allows for attribute access-like acces of its elements (so we can do `config.model_type = "CombinedINR"` instead of `config["model_type"] = "CombinedINR"`). You can also just use a dictionary instead.

Then we'll use the tools from `common_jax_utils` to first get a model from this config so we can inspect it, and then just run the experiment specified by the config.

Doing this in a config instead of hard coded might seem like extra work, but consider this:
1. you can serialize this config as a json file or a yaml file to later get the same model and experimental settings back 
   so when you are experimenting with different architectures, if you just store the configs you've used, you can easily recreate previous results
2. when we get to running hyper parameter sweeps, you can easily get these configs (with a pick for the varying hyper parameters) from wandb
   and then run an experiment specified by that config on any machine you want, e.g. on Snellius

In [2]:
config = Config()

# first we specify what the model should look like
config.architecture = './model_components'  # module containing all relevant classes for architectures
# NB if the classes relevant for creating the model are spread over multiple modules, this is no problem
# let config.architecture be the module that contains the "main" model class, and for all other components just specify the module
# or specify the other modules as default modules to the tools in common_jax_utils.run_utils
config.model_type = 'inr_modules.NeRF.from_config'

config.model_config = dict(
    in_size=(3, 3),
    out_size=(1, 3),
    bottle_size=256,
    block_length=4, 
    block_width=256,
    num_blocks=2,
    condition_length=1,
    condition_width=256,
    layer_type='inr_layers.SirenLayer',
    activation_kwargs={'w0': 30.},
    initialization_scheme=None,
    initialization_scheme_kwargs=None,
    positional_encoding_layer=('inr_layers.ClassicalPositionalEncoding.from_config', {'num_frequencies':6}),
    direction_encoding_layer=('inr_layers.ClassicalPositionalEncoding.from_config', {'num_frequencies':6}),
)


# next, we set up the training loop, including the 'target_function' that we want to mimic
config.trainer_module = './inr_utils/'  # similarly to config.architecture above, here we just specify in what module to look for objects by default
# config.trainer_type = 'training.train_inr'
# config.trainer_type = 'training.train_inr_with_dataloader'
config.trainer_type = 'training.train_with_dataloader_scan'

config.num_cycles = 50
config.steps_per_cycle = 1000

config.loss_evaluator = 'losses.NeRFLossEvaluator'
config.loss_evaluator_config = dict(
    #state_update_function = ('state_test_objects.py', 'counter_updater'),
    num_coarse_samples=128,
    num_fine_samples=512,
    near=-1.,
    far=1.,
    noise_std=1.,  # this seems to be a reasonable value, based on https://github.com/bmild/nerf/blob/18b8aebda6700ed659cb27a0c348b737a5f6ab60/config_fern.txt#L14
    white_bkgd=True,
    lindisp=False,
    randomized=True,
    parallel_batch_size=50, # was 500
)



# config.sampler = ('sampling.NeRFSyntheticScenesSampler',{
#     'split': 'train', 
#     'name': 'vase',
#     'batch_size': 10,
#     'poses_per_batch': 10,
#     'base_path': 'synthetic_scenes',
#     'size_limit': 100
# })
config.dataloader = 'nerf_utils.SyntheticScenesDataLoader'
config.dataloader_config = {
    'split': 'train', 
    'name': 'vase',
    'batch_size': 50, # was 500
    'poses_per_batch': 10, # was 100
    'base_path': 'example_data/synthetic_scenes',
    'size_limit': -1
}

config.optimizer = 'adamw'  # we'll have to add optax to the additional default modules later
config.optimizer = 'training.OptimizerFactory.single_optimizer'#'adamw'  # we'll have to add optax to the additional default modules later
config.optimizer_type = 'adamw'
config.optimizer_config = {
    'learning_rate': 5e-4,#1.5e-4
    'weight_decay': 1e-5,
}
config.optimizer_mask = 'masking.array_mask'
# config.steps = 20000 #changed from 40000
config.use_wandb = True

# now we want some extra things, like logging, to happen during training
# the inr_utils.training.train_inr function allows for this through callbacks.
# The callbacks we want to use can be found in inr_utils.callbacks
# config.after_step_callback = 'callbacks.ComposedCallback'
# config.after_step_callback_config = {
#     'callbacks':[
#         ('callbacks.print_loss', {'after_every':100}),  # only print the loss every 400th step
#         'callbacks.report_loss',  # but log the loss to wandb after every step
#         # ('callbacks.MetricCollectingCallback', # this thing will help us collect metrics and log images to wandb
#         #      {'metric_collector':'metrics.MetricCollector'}
#         # ),
#         'callbacks.raise_error_on_nan'  # stop training if the loss becomes NaN
#     ],
#     'show_logs': False
# }

config.after_cycle_callback = 'callbacks.ComposedCallback'
config.after_cycle_callback_config = {
    'callbacks':[
        ('callbacks.print_loss', {'after_every':1}),  # only print the loss every 400th step
        'callbacks.report_loss',  # but log the loss to wandb after every step
        # ('callbacks.MetricCollectingCallback', # this thing will help us collect metrics and log images to wandb
        #      {'metric_collector':'metrics.MetricCollector'}
        # ),
        'callbacks.raise_error_on_nan'  # stop training if the loss becomes NaN
    ],
    'show_logs': False
}

config.after_training_callback = None

    # num_coarse_samples=128,
    # num_fine_samples=512,
    # near=-1.,
    # far=1.,
    # noise_std=1.,  # this seems to be a reasonable value, based on https://github.com/bmild/nerf/blob/18b8aebda6700ed659cb27a0c348b737a5f6ab60/config_fern.txt#L14
    # white_bkgd=True,
    # lindisp=False,
    # randomized=True,
    # parallel_batch_size=500,

config.metric_collector_config = {  # the metrics for MetricCollectingCallback / metrics.MetricCollector
    'metrics':[
        ('metrics.ViewSynthesisComparison', {
            'frequency':'every_n_batches',
            'split':'train',
            'name':'vase',
            'num_coarse_samples':128,
            'num_fine_samples':512,
            'near':-1.,
            'far':1.,
            'noise_std':1.,
            'white_bkgd':True,
            'lindisp':False,
            'randomized':True,
            'folder':'synthetic_scenes',
        })


    ],
    'batch_frequency': 1,  # compute all of these metrics every 400 batches
    'epoch_frequency': 1  # not actually used
}


#config.after_training_callback = ('state_test_objects.py', 'after_training_callback')

# config.metric_collector_config = {  # the metrics for MetricCollectingCallback / metrics.MetricCollector
#     'metrics':[
#         ('metrics.PlotOnGrid2D', {'grid': 256, 'batch_size':8*256, 'frequency':'every_n_batches'}),  
#         # ^ plots the image on this fixed grid so we can visually inspect the inr on wandb
#         ('metrics.MSEOnFixedGrid', {'grid': [2040, 1356], 'batch_size':2040, 'frequency': 'every_n_batches'})
#         # ^ compute the MSE with the actual image pixels
#     ],
#     'batch_frequency': 400,  # compute all of these metrics every 400 batches
#     'epoch_frequency': 1  # not actually used
# }

#config.after_training_callback = None  # don't care for one now, but you could have this e.g. store some nice loss plots if you're not using wandb 
config.optimizer_state = None  # we're starting from scratch

In [3]:
# let's first see if we get the correct model
try:
    inr = cju.run_utils.get_model_from_config_and_key(
        prng_key=next(key_gen),
        config=config,
        model_sub_config_name_base='model',
        add_model_module_to_architecture_default_module=False, # since the model is already in the default module specified by 'architecture',
    )
except Exception as e:
    traceback.print_exc()
    print(e)
    print('\n')
    pdb.post_mortem()

In [4]:
inr

NeRF(
  coarse_model=NeRFComponent(
    block_pos_enc=ClassicalPositionalEncoding(_embedding_matrix=f32[6]),
    blocks=Sequential(
      layers=(
        NeRFBlock(
          net=Sequential(
            layers=(
              SirenLayer(
                weights=f32[256,39],
                biases=f32[256],
                activation_kwargs={'w0': 30.0},
                learnable_kwarg_keys=()
              ),
              SirenLayer(
                weights=f32[256,256],
                biases=f32[256],
                activation_kwargs={'w0': 30.0},
                learnable_kwarg_keys=()
              ),
              SirenLayer(
                weights=f32[256,256],
                biases=f32[256],
                activation_kwargs={'w0': 30.0},
                learnable_kwarg_keys=()
              ),
              SirenLayer(
                weights=f32[256,256],
                biases=f32[256],
                activation_kwargs={'w0': 30.0},
                learnable_kwarg_keys=

In [5]:
# next we get the experiment from the config using common_jax_utils.run_utils.get_experiment_from_config_and_key
experiment = cju.run_utils.get_experiment_from_config_and_key(
    prng_key=next(key_gen),
    config=config,
    model_kwarg_in_trainer='inr',
    model_sub_config_name_base='model',  # so it looks for "model_config" in config
    trainer_default_module_key='trainer_module',  # so it knows to get the module specified by config.trainer_module
    additional_trainer_default_modules=[optax],  # remember the don't forget to add optax to the default modules? This is that 
    add_model_module_to_architecture_default_module=False,
    initialize=False  # don't run the experiment yet, we want to use wandb
)

In [6]:
experiment

PostponedInitialization(cls=train_with_dataloader_scan, kwargs={'loss_evaluator': PostponedInitialization(cls=NeRFLossEvaluator, kwargs={'num_coarse_samples': 128, 'num_fine_samples': 512, 'near': -1.0, 'far': 1.0, 'noise_std': 1.0, 'white_bkgd': True, 'lindisp': False, 'randomized': True, 'parallel_batch_size': 50, 'state_update_function': None}, missing_args=[]), 'dataloader': PostponedInitialization(cls=SyntheticScenesDataLoader, kwargs={'split': 'train', 'name': 'vase', 'batch_size': 50, 'poses_per_batch': 10, 'base_path': 'example_data/synthetic_scenes', 'size_limit': -1, 'key': Array([1722811570, 3826931003], dtype=uint32)}, missing_args=[]), 'optimizer': PostponedInitialization(cls=single_optimizer, kwargs={'optimizer_type': <function adamw at 0x7fdf0651ee60>, 'optimizer_config': {'learning_rate': 0.0005, 'weight_decay': 1e-05}, 'optimizer_mask': <function array_mask at 0x7fdeec515f30>, 'learning_rate_schedule': None, 'schedule_boundaries': None}, missing_args=[]), 'steps_per_cy

In [ ]:
# and we run the experiment while logging things to wandb
try:
    with wandb.init(
        project='inr_edu_24',
        notes='test',
        tags=['test']
    ) as run:
        results = experiment.initialize()
except Exception as e:
    print(e)
    print()
    traceback.print_exc()
    pdb.post_mortem()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [8]:
inr, losses, optimizer_state, state, loss_evaluator, additional_output = results
inr

NameError: name 'results' is not defined

In [ ]:
from state_test_objects import after_training_callback, CountingIdentity
after_training_callback(losses, inr, state)

In [ ]:
inr.terms[0].layers[0]

In [ ]:
isinstance(inr.terms[0].layers[0], CountingIdentity)

In [ ]:
type(inr.terms[0].layers[0])